# Nikola

### Static site generator

# Features

* Static sites: it’s just a bunch of HTML files and assets.
* Incremental builds/rebuild using `doit`, so Nikola is fast.
* Multilingual
* Extensible
* Friendly CLI

* Multiple input formats such as reStructuredText, Markdown, HTML and **Jupyter Notebooks** (out of the box as part of the core!!)

# The core of the Nikola / Jupyter integration

* https://github.com/getnikola/nikola/blob/master/nikola/plugins/compile/ipynb.py

```python
from nbconvert.exporters import HTMLExporter

...

def _compile_string(self, nb_json):
    """Export notebooks as HTML strings."""
    self._req_missing_ipynb()
    c = Config(self.site.config['IPYNB_CONFIG'])
    c.update(get_default_jupyter_config())
    exportHtml = HTMLExporter(config=c)
    body, _ = exportHtml.from_notebook_node(nb_json)
    return body

```

# Some other gems

```python
def read_metadata(self, post, lang=None):
    """Read metadata directly from ipynb file.
    As ipynb files support arbitrary metadata as json, the metadata used by Nikola
    will be assume to be in the 'nikola' subfield.
    """
    self._req_missing_ipynb()
    if lang is None:
        lang = LocaleBorg().current_lang
    source = post.translated_source_path(lang)
    with io.open(source, "r", encoding="utf8") as in_file:
        nb_json = nbformat.read(in_file, current_nbformat)
    # Metadata might not exist in two-file posts or in hand-crafted
    # .ipynb files.
    return nb_json.get('metadata', {}).get('nikola', {})
```

```python
def create_post(self, path, **kw):
    """Create a new post."""
    self._req_missing_ipynb()
    content = kw.pop('content', None)
    onefile = kw.pop('onefile', False)
    kernel = kw.pop('jupyter_kernel', None)
    # is_page is not needed to create the file
    kw.pop('is_page', False)

    metadata = {}
    metadata.update(self.default_metadata)
    metadata.update(kw)

    makedirs(os.path.dirname(path))

    if content.startswith("{"):
        # imported .ipynb file, guaranteed to start with "{" because it’s JSON.
        nb = nbformat.reads(content, current_nbformat)
    else:
        nb = nbformat.v4.new_notebook()
        nb["cells"] = [nbformat.v4.new_markdown_cell(content)]

        if kernel is None:
            kernel = self.default_kernel
            self.logger.notice('No kernel specified, assuming "{0}".'.format(kernel))

        IPYNB_KERNELS = {}
        ksm = kernelspec.KernelSpecManager()
        for k in ksm.find_kernel_specs():
            IPYNB_KERNELS[k] = ksm.get_kernel_spec(k).to_dict()
            IPYNB_KERNELS[k]['name'] = k
            del IPYNB_KERNELS[k]['argv']

        if kernel not in IPYNB_KERNELS:
            self.logger.error('Unknown kernel "{0}". Maybe you mispelled it?'.format(kernel))
            self.logger.info("Available kernels: {0}".format(", ".join(sorted(IPYNB_KERNELS))))
            raise Exception('Unknown kernel "{0}"'.format(kernel))

        nb["metadata"]["kernelspec"] = IPYNB_KERNELS[kernel]

    if onefile:
        nb["metadata"]["nikola"] = metadata

    with io.open(path, "w+", encoding="utf8") as fd:
        nbformat.write(nb, fd, 4)
```        

# Let see it in action!

In [3]:
cd damian_blog/

/media/data/devel/damian_blog


In [7]:
!ls -la

total 9480
drwx------ 13 damian damian    4096 jul 12 14:16 .
drwxrwxr-x 24 damian damian    4096 ago 25 01:28 ..
drwxrwxr-x  6 damian damian    4096 jun 12 19:49 cache
-rw-rw-r--  1 damian damian   41009 jun 12 19:45 conf.py
-rw-rw-r--  1 damian damian    1607 may 10  2015 Customization.md
-rw-rw-r--  1 damian damian   80632 jul 12 14:15 .doit.db.bak
-rw-rw-r--  1 damian damian 9375377 jul 12 14:15 .doit.db.dat
-rw-rw-r--  1 damian damian   80632 jul 12 14:15 .doit.db.dir
drwx------  3 damian damian    4096 ago 28  2013 files
drwx------ 13 damian damian    4096 jul 12 13:52 galleries
drwx------  9 damian damian    4096 jul 12 14:16 .git
-rwxr-xr-x  1 damian damian     343 sep 24  2013 Guardfile
drwxr-xr-x  2 damian damian    4096 jun 19  2016 .ipynb_checkpoints
-rw-rw-r--  1 damian damian   31785 may 31  2015 old_conf.py
drwxrwxr-x 14 damian damian    4096 jun 13 08:45 output
drwxrwxr-x  3 damian damian    4096 may 31  2015 plugins
drwx------  3 damian damian   12288 jul 12 14:15 post

In [ ]:
title = "We are above 1000 stars!"

In [ ]:
tags_list = ['Jupyter', 'python', 'reveal', 'RISE', 'slideshow']

In [ ]:
tags = ', '.join(tags_list)

In [ ]:
!nikola new_post -f ipynb -t "{title}" --tags="{tags}"

```
Creating New Post
-----------------

Title: We are above 1000 stars!
Scanning posts......done!
[2017-07-12T16:45:00Z] NOTICE: compile_ipynb: No kernel specified, assuming "python3".
[2017-07-12T16:45:01Z] INFO: new_post: Your post's text is at: posts/we-are-above-1000-stars.ipynb
```

In [ ]:
!nikola build

In [ ]:
!nikola deploy

In [20]:
from IPython.display import IFrame
IFrame("http://www.damian.oquanta.info/", 980, 600)